In [1]:
import numpy as np
import astropy.units as u

%matplotlib inline

In [2]:
def foo(y):
    return -0.337 - (0.064 * y**2)

def betaOfMu(mu):
    return 10**foo(np.log10(mu * 1e15))

In the [Chernoff et al 2020 paper](https://ui.adsabs.harvard.edu/abs/2020MNRAS.491..596C/abstract) equation 12 gives the microlensing rate per star per second: 

$\frac{\Gamma}{N_\star} = \int_0^D dr' dl \left(\frac{dn}{dl}\right)'\langle \frac{dA_\perp}{dt} \rangle$.

In the paragraph just above, the authors state that $\langle \frac{dA_\perp}{dt} \rangle \sim lc$.

I take $\left(\frac{dn}{dl}\right)'$ to be $\frac{1}{l}\mathcal{F}(r')\,\mathcal{G}\,\left(\frac{dn}{d\ln{l}}\right)_\text{base}$, as in Equation 3, which invokes clustering via $\mathcal{F}$. In their paper, $\mathcal{G} = 10^2$ as stated in the first paragraph on page 598, and equation 1 states that:

$\left(\frac{dn}{d\ln{l}}\right)_\text{base} = 1.15 \times 10^{-6} \frac{x}{(1+x)^{5/2}} \frac{f_{0.2}\alpha_{0.1}^{1/2}}{(\Gamma_{50}\mu_{-13})^{3/2}} \text{kpc}^{-3}$,

where $\mu_{-13} = \frac{G \mu}{c^2} \times 10^{13}$, $x = \frac{l}{l_g}$, $l_g = 0.0206 \, \Gamma_{50} \, \mu_{-13}$ pc, and $\Gamma_{50} = f_{0.2} = \alpha_{0.1} = 1$. The clustering enhancement $\mathcal{F}(r) = \beta(\mu)\mathcal{E}(r) = \beta(\mu) \frac{\rho_{DM}(r)}{\Omega_{DM} \rho_{\text{crit}}}$ is defined in the second paragraph on the right side of page 599. The functional form of $\beta(\mu)$ is given in Appendix F of [Chernoff and Tye's 2018 paper](https://arxiv.org/pdf/1712.05060.pdf). $\mathcal{E} = \frac{\rho_{DM}(r)}{\Omega_{DM} \rho_{\text{crit}}}$ is defined in equation 5: 

$ \rho_{\text{MW - M31}} = \begin{cases}
\frac{A}{r^{9/4}} & 0 < r < r_1 \\ 
\frac{2^{3/4}A}{(B-r)^{9/4}} & r_1 < r < B
\end{cases} $

where $A = 1.15 \times 10^9 M_\odot / \text{kpc}^{3/4}$, $r_1 = 345$ kpc, and $B = 780$ kpc. I use $\Omega_{DM} = 1/4$ and $\rho_{\text{crit}} = 2.7754 \times 10^{11} \, h^2 \, M_\odot / \text{Mpc}^3$ with $h=0.7$. Finally, I place the observer 8 kpc from the Milky Way center and the source 8 kpc in front of the center of M31, as indicated in table 1 of the 2020 paper.

Combining all this gives:

$\frac{\Gamma}{N_\star} = \int_8^{772}dr' \int_0^\infty dl \, \frac{1}{l}\, \beta(\mu) \, \frac{1}{\Omega_{DM}\, \rho_{\text{crit}}} \,\left( \begin{cases}
\frac{A}{r^{9/4}} & 0 < r < r_1 \\ 
\frac{2^{3/4}A}{(B-r)^{9/4}} & r_1 < r < B
\end{cases}\right) \, \mathcal{G}\, 1.15 \times 10^{-6} \, \frac{x}{(1+x)^{5/2}} \, \frac{f_{0.2}\alpha_{0.1}^{1/2}}{(\Gamma_{50}\mu_{-13})^{3/2}} \, \text{kpc}^{-3}  lc$

Cancelling the factors of $l$ and using $dl = l_g dx$ gives:

$\frac{\Gamma}{N_\star} = \int_8^{772}dr' \int_0^\infty dx l_g \, \beta(\mu) \, \frac{c}{\Omega_{DM}\, \rho_{\text{crit}}} \,\left( \begin{cases}
\frac{A}{r^{9/4}} & 0 < r < r_1 \\ 
\frac{2^{3/4}A}{(B-r)^{9/4}} & r_1 < r < B
\end{cases}\right) \, \mathcal{G}\, 1.15 \times 10^{-6} \, \frac{x}{(1+x)^{5/2}} \, \frac{f_{0.2}\alpha_{0.1}^{1/2}}{(\Gamma_{50}\mu_{-13})^{3/2}} \, \text{kpc}^{-3} $.

The $r$ integral gives:

$\int_8^{772} dr \begin{cases}
\frac{A}{r^{9/4}} & 0 < r < r_1 \\ 
\frac{2^{3/4}A}{(B-r)^{9/4}} & r_1 < r < B
\end{cases} = A \, (0.059 + 0.1)$ kpc$^{-5/4}$

the $x$ integral gives:

$\int_0^\infty dx \frac{x}{(1+x)^{(5/2)}} = 4/3$.

Finally we have

$\frac{\Gamma}{N_\star} = l_g \, \beta(\mu) \, \frac{c}{\Omega_{DM}\, \rho_{\text{crit}}} \, \left[A \, (0.059 + 0.1) \text{kpc}^{-5/4} \right]\, 
\mathcal{G} \, 1.15 \times 10^{-6} \, \frac{4}{3} \, \frac{f_{0.2}\alpha_{0.1}^{1/2}}{(\Gamma_{50}\mu_{-13})^{3/2}} \, \text{kpc}^{-3} $

In [3]:
f02 = 1
a01 = 1
gamma50 = 1

tension = 1e-9
mu13 = tension * 1e13
lg = 0.0206 * gamma50 * mu13 * u.pc
beta = betaOfMu(tension)
speedOfLight = 2.98e8 * u.m / u.s
omegaDM = 1/4
littleHubble = 0.7
rhoCritical = 2.7754e11 * littleHubble**2 * u.solMass / u.megaparsec**3
aFactor = 1.15e9 * u.solMass / u.kpc**(3/4)
rIntegral = (0.059 + 0.1) * u.kpc**(-5/4)
gFactor = 1e2
xIntegral = 4/3

tObs = 1 * u.yr 

In [4]:
eventsPerStarPerTime = (lg * beta * (speedOfLight / (omegaDM * rhoCritical))
                        * aFactor * rIntegral * gFactor * 1.15e-6 * (4/3) * 
                        (f02 * np.sqrt(a01) / (gamma50 * mu13)**(3/2)) * u.kpc**(-3))
(eventsPerStarPerTime * tObs).decompose()

<Quantity 1.18333245e-10>